In [1]:
import pandas as pd
import ray
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import plotly.graph_objects as go
from itertools import combinations
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import (train_test_split, GridSearchCV,
                                    StratifiedKFold, StratifiedShuffleSplit,
                                    cross_val_score)
from sklearn.metrics import make_scorer, recall_score, precision_score, f1_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier
import numpy as np
import joblib
from pandarallel import pandarallel
from sklearn.preprocessing import LabelEncoder, StandardScaler
import re

2023-10-26 11:37:22.381090: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 11:37:22.407870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-26 11:37:22.407890: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-26 11:37:22.407906: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-26 11:37:22.412526: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 11:37:22.413118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
def busqueda(expression,column,name=None):
    if column is np.nan:
        return np.nan
    m = re.search(expression,column)
    if m == None:
        return np.nan
    if name != None:
        return name
    else :
        return m.group(0)

In [43]:
df_1 = pd.read_csv("/home/marcelo/nico/MasivasData/Skiff_zari_gaia_OGLE_Splus_Unwise_vsxpredicted.csv")

/tmp/ipykernel_5899/1297598039.py:1: DtypeWarning: Columns (72,74,83,84,85,86,88,89,105,116,119,122,124,125,127,128,130,131,133,134,141,163,166,167,186,193,211,279,280,285) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv("/home/marcelo/nico/MasivasData/Skiff_zari_gaia_OGLE_Splus_Unwise_vsxpredicted.csv")


In [4]:
skiff = df_1[['source_id_1',"skiff_type",'Bibcode','GroupID_2arcsec_skiff']].astype(str)

In [5]:
skiff['skiff_type'] = skiff['skiff_type'].replace('nan', np.nan)
skiff['Bibcode'] = skiff['Bibcode'].replace('nan', np.nan)
skiff['GroupID_2arcsec_skiff'] = skiff['GroupID_2arcsec_skiff'].replace('nan', np.nan)

In [6]:
skiff = skiff.loc[skiff['skiff_type'].notna()]

In [7]:
skiff["skiff_mk"] = skiff.apply(lambda row: busqueda(r"^(?!.*[+/])[a-zA-Z]\d+(?:\.\d+)?",row["skiff_type"]),axis=1)

In [8]:
skiff.loc[(skiff['skiff_mk'].isna())&(skiff['skiff_type'].str[:2]=='em'),'skiff_mk'] = 'em_WR_Be'
skiff.loc[skiff['skiff_mk'].isna(),'skiff_mk'] = skiff.loc[skiff['skiff_mk'].isna()].apply(lambda row: busqueda(r"WR|WN|WC", row["skiff_type"],'em_WR_Be'), axis=1)
skiff.loc[skiff['skiff_mk'].isna(),'skiff_mk'] = skiff.loc[skiff['skiff_mk'].isna()].apply(lambda row: busqueda(r"Be", row["skiff_type"],'em_WR_Be'), axis=1)
skiff.loc[skiff['skiff_mk'].isna(),'skiff_mk'] = skiff.loc[skiff['skiff_mk'].isna()].apply(lambda row: busqueda(r"OB", row["skiff_type"]), axis=1)

In [9]:
skiff.loc[(skiff['skiff_mk'].isna())&(skiff['skiff_type'].str[0]=='h'),'skiff_mk'] = skiff.loc[(skiff['skiff_mk'].isna())&(skiff['skiff_type'].str[0]=='h')]['skiff_type'].str[1:3]

In [10]:
def busqueda_2(pattern, string, default=None):
    match = re.search(pattern, string)
    if match:
        return match.group(1)  # Retornamos el primer grupo de captura
    return default

skiff.loc[skiff['skiff_mk'].isna(),'skiff_mk'] = skiff.loc[skiff['skiff_mk'].isna()].apply(lambda row: busqueda_2(r"^(.*?)(?=(V|IV|III|II|Ia|Ib|Iab)(?:/(V|IV|III|II|Ia|Ib|Iab))?$)", row["skiff_type"]), axis=1)


In [11]:
skiff.loc[skiff['skiff_mk'].isna(),'skiff_mk'] = skiff.loc[skiff['skiff_mk'].isna()].apply(lambda row: busqueda(r"^(O|B|A|F|G|K|M)$", row["skiff_type"]), axis=1)


In [12]:
skiff.loc[(skiff['skiff_type']=='Ofe'),'skiff_mk'] = 'em_WR_Be'

In [13]:
skiff.loc[skiff['skiff_mk'].isna(),'skiff_mk'] = skiff.loc[skiff['skiff_mk'].isna()].apply(lambda row: busqueda(r"e", row["skiff_type"],'em_WR_Be'), axis=1)


In [14]:
skiff.loc[skiff['skiff_mk'].isna(),'skiff_mk'] = skiff.loc[skiff['skiff_mk'].isna()].apply(lambda row: busqueda(r"^(O|B|A|F|G|K|M|N)(\d*(\.\d*)?)?(\/(\d*(\.\d*)?)?)?", row["skiff_type"]), axis=1)

In [15]:
skiff_mk = skiff.loc[(skiff['skiff_mk']!='em_WR_Be')&(skiff['skiff_mk']!='OB')&(skiff['skiff_mk'].notna())]

In [16]:
skiff_mk['skiff_mk'] = skiff_mk['skiff_mk'].str.replace('.', '')

/tmp/ipykernel_5899/2992802417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skiff_mk['skiff_mk'] = skiff_mk['skiff_mk'].str.replace('.', '')


In [17]:
replace_map = {
    'O': '0.',
    'B': '1.',
    'A': '2.',
    'F': '3.',
    'G': '4.',
    'K': '5.',
    'M': '6.'
}

In [18]:
skiff_mk['skiff_mk'] = skiff_mk['skiff_mk'].replace(replace_map, regex=True)


/tmp/ipykernel_5899/2936214468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skiff_mk['skiff_mk'] = skiff_mk['skiff_mk'].replace(replace_map, regex=True)


In [19]:
import re

def extract_first_float(s):
    match = re.search(r"(\d{1,2}(?:\.\d{1,2})?)", s)
    return float(match.group()) if match else None

s = '0.97Ib + 0.9'
number = extract_first_float(s)
print(number)


0.97


In [20]:
skiff_mk = skiff_mk[skiff_mk['skiff_mk'].str.split('/').str[0].str.len() !=1].reset_index(drop=True)

In [21]:
n1 = skiff_mk['skiff_mk'].str.split('/').str[0].reset_index(drop=True)

In [22]:
n1 = n1.apply(extract_first_float)

In [23]:
entero = n1.astype(str).str.split('.').str[0]
decimal = n1.astype(str).str.split('.').str[1]

In [24]:
n2 = skiff_mk['skiff_mk'].str.split('/').str[1].replace(np.nan,0).replace('',0).astype(str).reset_index(drop=True)

In [25]:
decimal  = '0.'+decimal
n2  = '0.'+n2

In [26]:
n2 = n2.apply(extract_first_float)
decimal = decimal.apply(extract_first_float)

In [27]:
n2.loc[n2 > 0] = n2.loc[n2 > 0] / 2
decimal.loc[n2 > 0] = decimal.loc[n2 > 0] / 2


In [28]:
skiff_mk['label'] = entero.astype(int) + (decimal + n2)

In [29]:
import numpy as np

# Define los posibles valores
values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Selecciona las filas donde la longitud de 'skiff_type' es 1
mask = skiff_mk['skiff_type'].str.len() == 1

# Suma el valor en 'skiff_mk' con un valor aleatorio de 'values' y asigna el resultado a 'label'
skiff_mk.loc[mask, 'label'] = skiff_mk.loc[mask, 'label'] + np.random.choice(values, size=sum(mask))



In [30]:
skiff_mk["year"] = skiff_mk.apply(lambda row: busqueda(r"\d{4}",row["Bibcode"]),axis=1)  

In [31]:
skiff_mk["arxiv"] = skiff_mk.apply(lambda row: busqueda(r"arXiv",row["Bibcode"]),axis=1)  

In [32]:
skiff_mk.loc[skiff_mk["arxiv"].notna(),"year"] = "20" + skiff_mk.loc[skiff_mk["arxiv"].notna()]["year"].astype("str").str[:2]

In [33]:
skiff_mk["year"] = pd.to_numeric(skiff_mk["year"],errors="coerce")

In [34]:
# si realmente es el mismo objeto
#keywoard de los papers S/N y/o R
# Pasar a numeros los tipos espectrales redondear!
# Escribir en la tesis como determinar tipos espectrales
stars_unicas = skiff_mk.loc[skiff_mk["GroupID_2arcsec_skiff"].isna()]
star_no_unicas = skiff_mk.loc[skiff_mk["GroupID_2arcsec_skiff"].notna()]
#df_1 = pd.concat([stars_unicas,star_no_unicas])

In [35]:
# Calcula la desviación estándar dentro de cada grupo
std_by_group = star_no_unicas.groupby("GroupID_2arcsec_skiff")["label"].transform("std")

# Agrega la columna al DataFrame
star_no_unicas["std_sp"] = std_by_group

/tmp/ipykernel_5899/396490030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  star_no_unicas["std_sp"] = std_by_group


In [36]:
star_no_unicas = star_no_unicas.loc[(star_no_unicas["std_sp"]<0.5)|(star_no_unicas["std_sp"].isna())]

In [37]:
mean_by_group = star_no_unicas.groupby("GroupID_2arcsec_skiff")["label"].transform("mean")

# Agrega la columna al DataFrame
star_no_unicas["label"] = mean_by_group

In [38]:
star_no_unicas = star_no_unicas.sort_values(by="year",ascending=False).drop_duplicates(subset="GroupID_2arcsec_skiff",keep='first')

In [39]:
skiff_mk = pd.concat([star_no_unicas,stars_unicas])

In [44]:
df_1

,source_id_1,ra_1_1,dec_1_1,ra_error,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmdec,...,PrimW1,PrimW2,FlagsW1,FlagsW2,f_FlagsW1,f_FlagsW2,Prim,GroupID,GroupSize,Separation
0,233646976817920640,62.929438,45.399682,0.010003,0.007647,0.667127,0.013129,50.815056,0.131989,-2.964688,...,True,True,0.0,0.0,0.0,0.0,True,1.0,4.0,0.051736
1,233646976817920640,62.929438,45.399682,0.010003,0.007647,0.667127,0.013129,50.815056,0.131989,-2.964688,...,True,True,0.0,0.0,0.0,0.0,True,1.0,4.0,0.051736
2,233646976817920640,62.929438,45.399682,0.010003,0.007647,0.667127,0.013129,50.815056,0.131989,-2.964688,...,True,True,0.0,0.0,0.0,0.0,True,1.0,4.0,0.051736
3,233646976817920640,62.929438,45.399682,0.010003,0.007647,0.667127,0.013129,50.815056,0.131989,-2.964688,...,True,True,0.0,0.0,0.0,0.0,True,1.0,4.0,0.051736
4,3440665079255783296,95.963683,34.949777,0.028062,0.027380,0.162584,0.032647,4.980018,0.256059,-1.424456,...,False,False,0.0,0.0,0.0,0.0,False,2.0,4.0,0.103058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180243,5321470325202411904,128.507836,-52.154300,0.018835,0.020499,0.263431,0.021107,12.480559,-4.582145,4.806116,...,True,True,34.0,32.0,0.0,0.0,True,NaN,NaN,0.081991
1180244,4306612880787921920,285.998597,6.537983,0.020446,0.019099,0.528764,0.024796,21.324427,0.609010,-4.342659,...,True,True,0.0,0.0,0.0,0.0,True,NaN,NaN,0.590170
1180245,5813630490211736064,264.508115,-65.756562,0.024764,0.025970,0.171369,0.038170,4.489561,-2.617784,-2.812394,...,True,True,0.0,0.0,0.0,0.0,True,NaN,NaN,0.288961
1180246,3099461469831899776,101.944628,-6.622163,0.029762,0.029760,0.112302,0.034121,3.291275,-0.018374,0.090998,...,True,True,0.0,0.0,0.0,0.0,True,NaN,NaN,0.162946


In [45]:
df_1.drop(columns=['Bibcode', 'GroupID_2arcsec_skiff'], inplace=True)


In [46]:
# Convierte la columna 'source_id_1' a int64 en ambas DataFrames
df_1['source_id_1'] = df_1['source_id_1'].astype(int)
skiff_mk['source_id_1'] = skiff_mk['source_id_1'].astype(int)

# Ahora realiza el merge
result = skiff_mk.merge(df_1, on=['source_id_1','skiff_type'], how='left')


In [51]:
result.to_csv("/home/marcelo/nico/MasivasData/skiff_2arcsec_internal_match_preprocesing.csv",index=False)

In [231]:
df['Flux_w2_over_error'] = df['FW2'] / df['e_FW2']
df['Flux_w1_over_error'] = df['FW1'] / df['e_FW1']